In [1]:
import pandas as pd
import csv
import os
import glob
import numpy as np
import os
from os import listdir
from os.path import isfile, join
import re
import argparse
import sys

In [2]:
command_line = False #false for jupyter notebook
transaction_file = ""
workingDir = ""
if command_line:
    #command line
    parser = argparse.ArgumentParser(description='Process datashop file.')
    parser.add_argument('-programDir', type=str, help='the component program directory')
    parser.add_argument('-workingDir', type=str, help='the component instance working directory')
    parser.add_argument("-node", nargs=1, action='append')
    parser.add_argument("-fileIndex", nargs=2, action='append')
    
    args, option_file_index_args = parser.parse_known_args()
   
    activity_file = args.fileIndex[0][1] 
    transaction_file = args.fileIndex[1][1]

    workingDir = args.workingDir

else:
    activity_file = 'oli_content.csv'
    transaction_file = 'trans.txt'
    workingDir = "."

In [3]:
df_oli = pd.read_csv(activity_file)
#iterate through activities
    #create variable for total_inputs
    #create dict_count of input: num
    #create dict_feedback of input : feedback
    #go through questions (stepname)
        #save question content and id
        #find matches of question id to step name from trans.txt
            #go through these matches
                #loc to inputs column
                #if input is not in dict_count, add input : 1
                #else input : num++
                
                #loc to feedback text
                #if input is not in dict_feedback, add input: eval
                #else leave be
                
                #increment total_inputs ++
                

In [4]:
tx = pd.read_csv(transaction_file,delimiter="\t", index_col='Row')
acts2 = df_oli["Activity"].unique()
acts = []
for x in acts2:
    for n in x.split("'"):
        if n != '[]' and n != '[' and n != ']':
            acts.append(n)


In [5]:
def get_list_of_questions_and_content(activity):
#     activity = "['" + activity + "']"
    lst_ids = []
    lst_content = []
    question_id = df_oli.loc[df_oli['Activity'] == activity, 'Question Texts'].values[0]
    for n in question_id.split("'"):
        #id and content
        if n[0].isalpha():
            #ids
            find = n.find("Question")
            if find != -1:
                lst_ids.append(n[:find-1])
            #content
            else:
                lst_content.append(n)
    return lst_ids, lst_content

In [15]:
activities = []
question_list = []
question_content_list = []
total_count_nums = []
total_input_counts = []
total_input_feedbacks = []
# step_names = tx['Step Name'].tolist() #from trans.txt
for x in acts2:
    ques_list, ques_content = get_list_of_questions_and_content(x) #from the csv
#     question_list.append(ques_list)
#     question_content_list.append(ques_content)
    i = 0
    #iterate through ques_list and find matches
    for q in ques_list:
        total_inputs = 0
        dict_count = {}
        dict_feedback = {}
        df_matching = tx[tx['Step Name'].str.contains(q)==True]
        step_matches = list(df_matching['Step Name'].index)
        #iterate through step_matches
        for match in step_matches:
            if isinstance(tx['Input'][match], str):
                #input_content = (tx['Input'][match].replace('<material>', " ")).replace('</material>', " ")
                input_content = re.sub('<[^<]+?>', '', tx['Input'][match])

            #go through counts
            if(input_content not in dict_count):
                dict_count[input_content] = 1
            else:
                dict_count[input_content] = dict_count[input_content] + 1

            #go through feedback/evaluation
            if isinstance(tx['Feedback Text'][match], str):
                input_feedback = re.sub('<[^<]+?>', '', tx['Feedback Text'][match])
            if(input_content not in dict_feedback):
                dict_feedback[input_content] = input_feedback

            total_inputs = total_inputs + 1
        
        activities.append(x)
        total_input_counts.append(dict_count)
        total_input_feedbacks.append(dict_feedback)
        total_count_nums.append(total_inputs)
        question_list.append(q)
        question_content_list.append(ques_content[i])
        i = i + 1

Correct: Data wrangling is not just for the purposes of analytics. Each time data is extracted, it should be cleaned and enriched. 
Incorrect: You should consider data integration in all cases.
Correct: Data wrangling is not just for the purposes of analytics. Each time data is extracted, it should be cleaned and enriched. 
Correct: Data wrangling is not just for the purposes of analytics. Each time data is extracted, it should be cleaned and enriched. 
Correct: Data wrangling is not just for the purposes of analytics. Each time data is extracted, it should be cleaned and enriched. 
Correct: Data wrangling is not just for the purposes of analytics. Each time data is extracted, it should be cleaned and enriched. 
Correct: Data wrangling is not just for the purposes of analytics. Each time data is extracted, it should be cleaned and enriched. 
Correct: Data wrangling is not just for the purposes of analytics. Each time data is extracted, it should be cleaned and enriched. 
Correct: Data 

In [10]:
#add percentage to totalinputcounts
total_input_counts_revised = []
index = 0
for t in total_input_counts:
    temp = {}
    for k, v in t.items():
        if(k not in temp):
            percentage = round((v/total_count_nums[index])*100,2)
            temp[k] = str(v) + " (" + str(percentage) + "%)"
    total_input_counts_revised.append(temp)
    index = index + 1
    

In [11]:
df_final = pd.DataFrame()
df_final['Activity'] = activities
df_final['Question Id'] = question_list
df_final['Total Observations'] = total_count_nums
df_final['Question Content'] = question_content_list
df_final['Input Count and Percentage'] = total_input_counts_revised
df_final['Input Evaluation'] = total_input_feedbacks

df_final = df_final[['Activity','Question Id', 'Total Observations', 'Question Content', 'Input Count and Percentage', 'Input Evaluation']]
                       
df_final.to_csv("error_report.csv",  mode='w', index = False)
# pd.set_option('display.max_rows', None)
display(df_final)

,Activity,Question Id,Total Observations,Question Content,Input Count and Percentage,Input Evaluation
0,['newa67cd633857544de9b4654ef26ecbab4'],newa67cd633857544de9b4654ef26ecbab4_1a,16,Why do we need to consider integration in the ...,{'Data integration is important as transformed...,{'Data integration is important as transformed...
1,['newa67cd633857544de9b4654ef26ecbab4'],af4aaa0623d84715860ca33ef6fc7442,21,The transformation process in the data wrangli...,{'You should assess the quality of the dataset...,{'You should assess the quality of the dataset...
2,['newa67cd633857544de9b4654ef26ecbab4'],b8d9c090866244a4baa47a82a08a5738,25,You typically work with a sample of your datas...,"{'Stratified Sampling.': '17 (68.0%)', 'Cluste...",{'Stratified Sampling.': 'Stratified Sampling....
3,['newe131dd72d65a44499220491554114a99'],a870b69be9774e399fe16081959ffcbf,0,Model interpretation is useful because:,{},{}
4,['newe131dd72d65a44499220491554114a99'],e2670f71cb114b31a1118d962ec4d43e,0,Model interpretability strategies include:,{},{}
...,...,...,...,...,...,...
106,['newb8957500e03349cc8051dbeb27aeb04b'],cf85600ef8f644c5b3b4e851323b7126,0,This clustering technique only considers the w...,{},{}
107,['newb8957500e03349cc8051dbeb27aeb04b'],a251adf58b3c4045a35eb01066a186f1,0,The receiver operating characteristic curve:,{},{}
108,['newb8957500e03349cc8051dbeb27aeb04b'],affb36746ef74c20ae94a9eed778c4eb,0,The R-squared explains the degree to which ind...,{},{}
109,['newc26dfd3238be4dd2a0ea4b8d2abb59c1'],a1e96072cf064738919faa7421ad49af,53,A Data Science team has been contracted by a h...,"{'An Analytic Objective.': '27 (50.94%)', 'A B...",{'An Analytic Objective.': 'An Analytic Object...
